<div class="alert alert-info">

An interactive version of this notebook is available on [Google Colab](https://colab.research.google.com/github/starsimhub/tbsimV2/blob/main/docs/tutorials/comprehensive_analyzer_plots_example_ss3.0.ipynb) or [Binder](https://mybinder.org/v2/gh/starsimhub/tbsimV2/HEAD?labpath=docs%2Ftutorials%2Fcomprehensive_analyzer_plots_example_ss3.0.ipynb).

If running in Google Colab, install TBsimV2 from GitHub first:

```python
%pip install -q git+https://github.com/starsimhub/tbsimV2.git
```

</div>


# Comprehensive Analyzer Plots Example
This notebook demonstrates all available plotting methods in the `DwtAnalyzer` class from the `tbsim` package. It is adapted from the original Python script for interactive exploration.

In [ ]:
import tbsim as mtb
import starsim as ss
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
TBS = mtb.TBS

## Build a TB simulation with dwell time analyzer

In [ ]:
def build_tbsim():
    sim_params = dict(
        start = ss.date('2013'),
        stop = ss.date('2016'),
        rand_seed=1,
        dt=ss.days(7),
    )
    pop = ss.People(n_agents=1000)
    tb_params = dict(
        beta=ss.rate_prob(0.0025),
        init_prev=ss.bernoulli(p=0.25),
        rel_sus_latentslow=0.1,
        dt=ss.days(7),
    )
    tb = mtb.TB(tb_params)
    net = ss.RandomNet(dict(n_contacts=ss.poisson(lam=5), dur=0))
    dwell_analyzer = mtb.DwtAnalyzer(adjust_to_unit=True, unit=1.0, scenario_name='comprehensive_plots_example')
    sim = ss.Sim(
        people=pop,
        networks=net,
        diseases=tb,
        pars=sim_params,
        analyzers=dwell_analyzer,
    )
    sim.pars.verbose = 0
    return sim

## Run the simulation and extract the analyzer

In [ ]:
print("Building and running TB simulation...")
sim_tb = build_tbsim()
sim_tb.run()
analyzer = sim_tb.analyzers[0]

## 1. Sankey Diagrams
Demonstrate Sankey diagrams for state transitions.

In [ ]:
# Basic Sankey diagram for all agents
analyzer.sankey_agents()


In [ ]:

# Sankey diagram with dwell times
analyzer.sankey_dwelltimes(subtitle="State Transitions with Dwell Times")


In [ ]:

# Sankey diagrams by age groups
analyzer.sankey_agents_by_age_subplots(bins=[0, 5, 15, 30, 50, 200], scenario="Age-stratified Analysis")


In [ ]:

# Sankey diagrams with even age ranges
analyzer.sankey_agents_even_age_ranges(number_of_plots=3, scenario="Even Age Distribution")

## 2. Network Graphs
Visualize state transition networks.

In [ ]:
analyzer.graph_state_transitions(subtitle="State Transition Network", colormap='tab20')


In [ ]:

analyzer.graph_state_transitions_curved(subtitle="Curved State Transitions", colormap='plasma')

## 3. Histograms and Distributions
Explore dwell time distributions.

In [ ]:
help(analyzer.histogram_with_kde)



## 4. Interactive Bar Charts
Interactive bar charts for state transitions and reinfections.

In [ ]:
analyzer.barchar_all_state_transitions_interactive(
    dwell_time_bins=[0, 30, 90, 180, 365, float('inf')],
    filter_states=['-1.0.None', '0.0.Latent Slow', '1.0.Latent Fast', '2.0.Active Presymp']
)



## 5. Stacked Bar Charts
Stacked bar charts for cumulative time and dwell time analysis.

In [ ]:
# stacked_bars_states_per_agent_static
analyzer.barchar_all_state_transitions_interactive(
    dwell_time_bins=[0, 30, 90, 180, 365, float('inf')],
    filter_states=[  '1.0.Latent Fast', '2.0.Active Presymp']
)


## 6. Custom Transition Analysis
Custom transition subplots.

In [ ]:
custom_transitions = {
    '-1.0.None': ['0.0.Latent Slow', '1.0.Latent Fast'],
    '0.0.Latent Slow': ['2.0.Active Presymp', '-1.0.None'],
    '1.0.Latent Fast': ['2.0.Active Presymp', '-1.0.None']
}
analyzer.subplot_custom_transitions(transitions_dict=custom_transitions)

## 7. Survival Analysis
Kaplan-Meier survival curve for dwell times.

In [ ]:
analyzer.plot_kaplan_meier(dwell_time_col='dwell_time')

## 8. Using DwtPlotter Directly
Demonstrate additional plots using the DwtPlotter class.

In [ ]:
file_path = analyzer.file_path
print(f'Generated data file: {file_path}')
plotter = mtb.DwtPlotter(file_path=file_path)
plotter.histogram_with_kde(subtitle="From Generated File")
plotter.sankey_agents(subtitle="From Generated File")

## 9. Post Processor Demonstration
Example usage of the DwtPostProcessor for multiple simulation results. (This is informational; actual usage requires multiple result files.)

In [ ]:
# Example usage (commented out):
# postproc = mtb.DwtPostProcessor(directory='results', prefix='Baseline')
# postproc.sankey_agents(subtitle="Aggregated Results")
# postproc.histogram_with_kde(subtitle="Aggregated Distributions")
# postproc.reinfections_percents_bars_interactive(
#     target_states=[0.0, 1.0],
#     scenario="Aggregated Reinfection Analysis"
# )